In [1]:
# Cell 1: Environment and Imports
import os
import sys
from dotenv import load_dotenv

# Add project to path
sys.path.append('.')

# Load environment
load_dotenv()

# Import our LSTM components
from config.settings import LSTMConfig, AstraDBConfig
from core.gate_networks import LSTMGateNetwork
from core.memory_manager import MemoryManager
from models.memory_item import MemoryItem
from models.rl_experience import RLExperience, RLState, RLAction, RLReward
from database.astra_connector import AstraDBConnector
from rl_training.gate_optimizer import GateOptimizer
from rl_training.reward_calculator import RewardCalculator
from utils.similarity import cosine_similarity
from utils.token_counter import estimate_tokens

# Validate environment
DATABASE_ID = os.getenv('ASTRA_DATABASE_ID')
REGION = os.getenv('ASTRA_REGION') 
TOKEN = os.getenv('ASTRA_TOKEN')
KEYSPACE = os.getenv('ASTRA_KEYSPACE', 'memory_db')

if not all([DATABASE_ID, REGION, TOKEN]):
    raise ValueError("Missing Astra DB credentials in .env file!")

print("✅ All LSTM components imported successfully")
print(f"🔗 Database: {DATABASE_ID}")
print(f"🌍 Region: {REGION}")
print(f"🔑 Keyspace: {KEYSPACE}")

✅ All LSTM components imported successfully
🔗 Database: cb844ee7-37f7-453e-9ece-e0720040b3df
🌍 Region: us-east1
🔑 Keyspace: memory_db


In [2]:
# Cell 2: Initialize LSTM System
async def initialize_lstm_system():
    """Initialize the complete LSTM memory system"""
    
    # Create configuration
    config = LSTMConfig()
    config.astra_db.endpoint = f"https://{DATABASE_ID}-{REGION}.apps.astra.datastax.com"
    config.astra_db.token = TOKEN
    config.astra_db.keyspace = KEYSPACE
    
    # Initialize memory manager
    memory_manager = MemoryManager(config)
    await memory_manager.initialize()
    
    # Initialize RL trainer
    rl_trainer = GateOptimizer(
        gate_network=memory_manager.gate_network,
        learning_rate=0.001
    )
    
    return memory_manager, rl_trainer, config

# Run initialization
memory_manager, rl_trainer, config = await initialize_lstm_system()
print("✅ LSTM system initialized successfully")
print(f"📊 Gate thresholds: {memory_manager.gate_network.get_thresholds()}")

INFO:core.memory_manager:Initializing Memory Manager...
INFO:database.astra_connector:Connecting to Astra DB: https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com
INFO:astrapy.data.database:findCollections
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db "HTTP/1.1 200 OK"
INFO:astrapy.data.database:finished findCollections
INFO:database.astra_connector:Found collections: []
INFO:database.astra_connector:Successfully connected to Astra DB
INFO:core.memory_manager:No existing gate network found, using fresh model
INFO:core.memory_manager:Memory Manager initialized successfully


✅ LSTM system initialized successfully
📊 Gate thresholds: {'forget': 0.30000001192092896, 'input': 0.4000000059604645, 'output': 0.4000000059604645}


In [3]:
# Cell 3: Test Memory Creation
import random
# Sample journal entries
sample_entries = [
    "I had an amazing meeting with Sarah today about the new AI project. Super excited about the possibilities!",
    "Feeling stressed about the upcoming deadline. Need to organize my time better.",
    "Learned something fascinating about neural networks today. The attention mechanism is incredible.",
    "Had lunch with my friend. We talked about his new job and he seems happy.",
    "Dentist appointment tomorrow at 2pm. Really nervous about it."
]

# Process entries through LSTM system
print("📝 Processing journal entries through LSTM gates...")
created_memories = []

for i, entry in enumerate(sample_entries, 1):
    print(f"\n📖 Entry {i}: '{entry[:50]}...'")
    
    # Generate dummy feature vector (replace with Component 4)
    feature_vector = [random.random() for _ in range(90)]
    
    # Generate dummy embeddings (replace with Component 3)  
    embeddings = [random.random() for _ in range(768)]
    
    # Process through memory manager
    memory = await memory_manager.process_new_entry(
        user_id="test_user_123",
        feature_vector=feature_vector,
        content=entry,
        embeddings=embeddings
    )
    
    if memory:
        created_memories.append(memory)
        print(f"✅ Memory created: {memory.memory_type} (importance: {memory.importance_score:.3f})")
    else:
        print("❌ Entry not stored (low importance)")

print(f"\n📊 Results: {len(created_memories)}/{len(sample_entries)} entries became memories")

INFO:core.memory_manager:Processing new entry for user test_user_123
INFO:core.memory_manager:Input gate score: 0.353, decision: False
INFO:core.memory_manager:Entry not important enough to create memory (score: 0.353)
INFO:core.memory_manager:Processing new entry for user test_user_123
INFO:core.memory_manager:Input gate score: 0.428, decision: True
INFO:astrapy.data.collection:insertOne on 'memory_embeddings'


📝 Processing journal entries through LSTM gates...

📖 Entry 1: 'I had an amazing meeting with Sarah today about th...'
❌ Entry not stored (low importance)

📖 Entry 2: 'Feeling stressed about the upcoming deadline. Need...'


INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished insertOne on 'memory_embeddings'
INFO:database.memory_store:Saved memory 07cb859e-b87f-4847-860e-cbd04f9fb68b for user test_user_123
INFO:core.memory_manager:Created new memory: 07cb859e-b87f-4847-860e-cbd04f9fb68b
INFO:core.memory_manager:Processing new entry for user test_user_123
INFO:core.memory_manager:Input gate score: 0.489, decision: True
INFO:astrapy.data.collection:insertOne on 'memory_embeddings'


✅ Memory created: emotion (importance: 0.428)

📖 Entry 3: 'Learned something fascinating about neural network...'


INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished insertOne on 'memory_embeddings'
INFO:database.memory_store:Saved memory 8e554e1d-856f-4b56-b8e4-b7616c9d6eaa for user test_user_123
INFO:core.memory_manager:Created new memory: 8e554e1d-856f-4b56-b8e4-b7616c9d6eaa
INFO:core.memory_manager:Processing new entry for user test_user_123
INFO:core.memory_manager:Input gate score: 0.387, decision: False
INFO:core.memory_manager:Entry not important enough to create memory (score: 0.387)
INFO:core.memory_manager:Processing new entry for user test_user_123
INFO:core.memory_manager:Input gate score: 0.443, decision: True
INFO:astrapy.data.collection:insertOne on 'memory_embeddings'


✅ Memory created: insight (importance: 0.489)

📖 Entry 4: 'Had lunch with my friend. We talked about his new ...'
❌ Entry not stored (low importance)

📖 Entry 5: 'Dentist appointment tomorrow at 2pm. Really nervou...'


INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished insertOne on 'memory_embeddings'
INFO:database.memory_store:Saved memory b7c3aff9-1fb9-48af-b8fa-515cf47eb312 for user test_user_123
INFO:core.memory_manager:Created new memory: b7c3aff9-1fb9-48af-b8fa-515cf47eb312


✅ Memory created: event (importance: 0.443)

📊 Results: 3/5 entries became memories


In [3]:
# Cell 4: Test Context Retrieval with debugging
import random
print("🔍 Testing memory context retrieval...")

try:
    # Test queries
    test_queries = [
        "How's work going?",
        "Tell me about AI", 
        "Any upcoming appointments?",
        "How am I feeling lately?"
    ]
    
    for query in test_queries:
        print(f"\n❓ Query: '{query}'")
        
        # Generate query features (placeholder)
        query_features = [random.random() for _ in range(90)]
        
        try:
            # Get relevant context
            relevant_memories, metadata = await memory_manager.get_relevant_context(
                user_id="test_user_123",
                query=query,
                query_features=query_features,
                max_tokens=1500
            )
            
            print(f"🎯 Found {len(relevant_memories)} relevant memories")
            print(f"📊 Metadata: {metadata}")
            
            if relevant_memories:
                for memory in relevant_memories[:2]:  # Show top 2
                    print(f"   💭 {memory.memory_type}: {memory.content_summary[:60]}...")
                    print(f"      Importance: {memory.importance_score:.3f}")
        
        except Exception as e:
            print(f"❌ Error processing query '{query}': {e}")
            import traceback
            traceback.print_exc()
            
except Exception as e:
    print(f"❌ Critical error: {e}")
    import traceback
    traceback.print_exc()

INFO:core.memory_manager:Assembling context for user test_user_123
INFO:astrapy.data.cursors.cursor:cursor fetching a page: (empty page state) from memory_embeddings


🔍 Testing memory context retrieval...

❓ Query: 'How's work going?'


INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.cursors.cursor:cursor finished fetching a page: (empty page state) from memory_embeddings
INFO:database.memory_store:Retrieved 3 memories for user test_user_123
INFO:core.memory_manager:Filtered to 3 relevant memories
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'


🎯 Found 3 relevant memories
📊 Metadata: {'query_tokens': 3, 'overhead_tokens': 200, 'available_tokens': 1297, 'memories_considered': 3, 'memories_selected': 3, 'token_utilization': 0.028, 'diversity_stats': {'emotion': 1, 'insight': 1, 'event': 1}, 'avg_relevance_score': 0.688806073864301, 'total_tokens_used': 42, 'reason': 'success'}
   💭 emotion: Feeling stressed about the upcoming deadline. Need to organi...
      Importance: 0.428
   💭 insight: Learned something fascinating about neural networks today. T...
      Importance: 0.489

❓ Query: 'Tell me about AI'


INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:core.memory_manager:Assembled context with 3 memories
INFO:core.memory_manager:Assembling context for user test_user_123
INFO:core.memory_manager:Filtered to 3 

🎯 Found 3 relevant memories
📊 Metadata: {'query_tokens': 4, 'overhead_tokens': 200, 'available_tokens': 1296, 'memories_considered': 3, 'memories_selected': 3, 'token_utilization': 0.028, 'diversity_stats': {'insight': 1, 'emotion': 1, 'event': 1}, 'avg_relevance_score': 0.698757673005263, 'total_tokens_used': 42, 'reason': 'success'}
   💭 insight: Learned something fascinating about neural networks today. T...
      Importance: 0.489
   💭 emotion: Feeling stressed about the upcoming deadline. Need to organi...
      Importance: 0.428

❓ Query: 'Any upcoming appointments?'


INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:core.memory_manager:Assembled context with 3 memories
INFO:core.memory_manager:Assembling context for user test_user_123
INFO:core.memory_manager:Filtered to 3 

🎯 Found 3 relevant memories
📊 Metadata: {'query_tokens': 4, 'overhead_tokens': 200, 'available_tokens': 1296, 'memories_considered': 3, 'memories_selected': 3, 'token_utilization': 0.028, 'diversity_stats': {'insight': 1, 'emotion': 1, 'event': 1}, 'avg_relevance_score': 0.7192286753654479, 'total_tokens_used': 42, 'reason': 'success'}
   💭 insight: Learned something fascinating about neural networks today. T...
      Importance: 0.489
   💭 emotion: Feeling stressed about the upcoming deadline. Need to organi...
      Importance: 0.428

❓ Query: 'How am I feeling lately?'


INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:astrapy.data.collection:updateOne on 'memory_embeddings'
INFO:httpx:HTTP Request: POST https://cb844ee7-37f7-453e-9ece-e0720040b3df-us-east1.apps.astra.datastax.com/api/json/v1/memory_db/memory_embeddings "HTTP/1.1 200 OK"
INFO:astrapy.data.collection:finished updateOne on 'memory_embeddings'
INFO:core.memory_manager:Assembled context with 3 memories


🎯 Found 3 relevant memories
📊 Metadata: {'query_tokens': 5, 'overhead_tokens': 200, 'available_tokens': 1295, 'memories_considered': 3, 'memories_selected': 3, 'token_utilization': 0.028, 'diversity_stats': {'insight': 1, 'emotion': 1, 'event': 1}, 'avg_relevance_score': 0.7404099132617313, 'total_tokens_used': 42, 'reason': 'success'}
   💭 insight: Learned something fascinating about neural networks today. T...
      Importance: 0.489
   💭 emotion: Feeling stressed about the upcoming deadline. Need to organi...
      Importance: 0.428


In [8]:
# Cell 5: RL Training Setup
from typing import List,Dict
print("🎯 Setting up RL Training...")

# Since created_memories from Cell 3 might not be available, let's get the actual memories
# that were successfully saved to the database
print("📥 Loading saved memories for RL training...")

try:
    # Get the memories that were actually saved
    saved_memories = await memory_manager._get_user_memories("test_user_123")
    print(f"✅ Found {len(saved_memories)} saved memories for RL training")
    
    if not saved_memories:
        print("❌ No memories found! Make sure Cell 3 ran successfully first")
    
except Exception as e:
    print(f"❌ Error loading memories: {e}")
    saved_memories = []

# Continue with the rest of your RL training code, using saved_memories instead of created_memories
def simulate_user_interaction(memories):
    """Simulate user interaction with memory context"""
    return {
        'response_length': random.randint(50, 200),
        'session_duration_minutes': random.uniform(2, 15),
        'has_follow_ups': random.choice([True, False]),
        'user_initiated': random.choice([True, False]),
        'memories_used': len(memories),
        'context_satisfaction': random.uniform(0.3, 1.0)
    }

print("🏆 Testing reward calculation...")

for i in range(3):
    # Simulate interaction - use saved_memories instead of created_memories
    interaction_data = simulate_user_interaction(saved_memories[:3])
    
    # Calculate reward using our reward calculator
    reward = reward_calc.calculate_simple_reward({
        'user_engagement': random.uniform(0.4, 1.0),
        'context_relevance': random.uniform(0.3, 1.0), 
        'memory_efficiency': random.uniform(0.5, 1.0),
        'user_satisfaction': random.uniform(0.4, 1.0),
        'conversation_quality': random.uniform(0.5, 1.0)
    })
    
    print(f"🎯 Interaction {i+1}: Reward = {reward:.3f}")
    print(f"   Engagement metrics: {interaction_data}")

print("✅ Reward calculation working")

🎯 Setting up RL Training...
📥 Loading saved memories for RL training...
✅ Found 3 saved memories for RL training
🏆 Testing reward calculation...
🎯 Interaction 1: Reward = 0.624
   Engagement metrics: {'response_length': 63, 'session_duration_minutes': 12.523259519955204, 'has_follow_ups': True, 'user_initiated': True, 'memories_used': 3, 'context_satisfaction': 0.8445545450570942}
🎯 Interaction 2: Reward = 0.674
   Engagement metrics: {'response_length': 192, 'session_duration_minutes': 12.747808981692172, 'has_follow_ups': False, 'user_initiated': True, 'memories_used': 3, 'context_satisfaction': 0.6646805459455181}
🎯 Interaction 3: Reward = 0.736
   Engagement metrics: {'response_length': 178, 'session_duration_minutes': 4.289956892674086, 'has_follow_ups': True, 'user_initiated': True, 'memories_used': 3, 'context_satisfaction': 0.3118299114559478}
✅ Reward calculation working
